# Deep Electromagnetic Sounding using Geomagnetic Observations

**IAGA Summer School 2025**, Lisbon, Portugal  
*Alexander Grayver*, University of Cologne 

### Method

We will implemented here the so-called **Geomagnetic Depth Sounding** method (also referred to as **Z/H**-method) to retrieve the 1-D electrical conductivity of the mantle under observatories. The method was first elaborated in [Banks 1969](https://doi.org/10.1111/j.1365-246X.1969.tb00252.x).

At periods longer than 2 days, the external magnetic field variations at mid latitudes are dominated by the magnetospheric ring current (RC) source. This source produces the magnetic field at the surface that can, to first-order, represented by the first zonal harmonic $P_1^0 (\cos \theta)$, where $\theta$ is the colatitude. Under this assumption about the geometry of the external geomagnetic field, a transfer function called $C$-response can be estimated at period $T$ as

$$
C(\omega; \sigma) = -\frac{a}{2}\tan(\theta)\frac{Z}{H},
$$

where $a = 6371.2$ km is the mean Earth's radius, $\omega = 2\pi/T$ is the angular frequency. $Z \equiv -\tilde{B}_r(\tilde{\theta},\tilde{\phi},\omega), H \equiv -\tilde{B}_{\theta}(\tilde{\theta},\tilde{\phi},\omega)$, are vertical and horizontal (North) magnetic field components expressed in centered dipole (geomagnetic) coordinates at a location on the Earth's surface characterized by geomagnetic co-latitude $\tilde{\theta}$ and longitude $\tilde{\phi}$. The $\sigma \equiv \sigma(r)$ is the electrical conductivity model of the subsurface, here assumed to vary only with the radial distance from the center.

Note that we assume that the magnetic field consists of only external and corresponding induced (internal) components. That is, the effect of all other time-varying magnetic field sources, such as the core, has been subtracted from $Z$ and $H$ prior to the analysis.

At this point, a few properties of the $C$-response can be mentioned (see [Weidelt 1972](https://www.mtnet.info/papers/PeterWeidelt/Weidelt_1972_ZGeophys.pdf) for more details):

- The period $T$ is proportional to the depth of sounding. Longer periods (smaller frequencies) attenuate slower, thus sounding deeper and vice versa.
- For a 1-D subsurface conductivity model, real part of $C$-response is the monotonically increasing function of period.
- For a 1-D subsurface conductivity model, imaginary part is of the same sign (never crosses zero). Its sign depends on the Fourier convention.

Apparent resistivity can be expressed from $C$-response as

$$
\rho^{app} = \omega\mu|C|^2
$$

For more details about $C$-responses and various practical aspects, refer to the literature:

- Banks, R. J. (1969). Geomagnetic variations and the electrical conductivity of the upper mantle. Geophysical Journal International, 17(5), 457-487.
- Weidelt, P. (1972). The Inverse Problem of Geomagnetic Induction, Zeitschriftfiir Geophysik, 1972, Band 38, Seite 257-289. Physica-Verfag, Wiirzburg
- Olsen, N. (1998). The electrical conductivity of the mantle beneath Europe derived from C-responses from 3 to 720 hr. Geophysical Journal International, 133(2), 298-308.
- Schmucker, U. (1999). A spherical harmonic analysis of solar daily variations in the years 1964–1965: response estimates and source fields for global induction—II. Results. Geophysical Journal International, 136(2), 455-476.
- Semenov, A., & Kuvshinov, A. (2012). Global 3-D imaging of mantle conductivity based on inversion of observatory C-responses—II. Data analysis and results. Geophysical Journal International, 191(3), 965-992.

## Downloading the time series

We will rely on the [ViresClient](https://viresclient.readthedocs.io/en/latest/) server that provides an access to quality-controlled database of ground observatory data delivered by the [INTERMAGNET](https://intermagnet.org/) and the [World Data Centre (WDC) for Geomagnetism](http://www.wdc.bgs.ac.uk/). Development of ViresClient is endorsed by the European Space Agency in the Earth Observation program space mission [Swarm](https://www.esa.int/Applications/Observing_the_Earth/FutureEO/Swarm).

### Import modules

In [ ]:
# Install dipole which will be used later
%pip install git+https://github.com/klaundal/dipole.git@8acb53

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings

import chaosmagpy.coordinate_utils as c_utils

from viresclient import SwarmRequest

%matplotlib inline

### Fetch the data from `Vires` server

First, setup a connection with the Vires server (note you need to login/register the first time you access the server):

In [ ]:
request = SwarmRequest()

Data are available as three collections: 1 second and 1 minute cadences, as well as specially derived hourly means over the past century. There are three data types AUX_OBSH (hour), AUX_OBSM (minute) and AUX_OBSS (second). For example, to access the hourly data, we need the collection name *SW_OPER_AUX_OBSH2_*. For other types, relevant collections can be requested:

In [ ]:
print(request.available_collections("AUX_OBSH", details=False))
print(request.available_collections("AUX_OBSM", details=False))
print(request.available_collections("AUX_OBSS", details=False))

Within each collection, the following variables are available:

In [ ]:
print(request.available_measurements("SW_OPER_AUX_OBSH2_"))
print(request.available_measurements("SW_OPER_AUX_OBSM2_"))
print(request.available_measurements("SW_OPER_AUX_OBSS2_"))

- `B_NEC` is the magnetic field vector in North (X), East (Y) and Center (Z) coordinate system
- `F` is total field intensity
- `IAGA_code` gives the official three-letter [IAGA INTERMAGNET](https://intermagnet.org/metadata/#/imos) codes that identify each observatory
- `Quality` is either "D" or "Q" to indicate whether data is definitive (D) or quasi-definitive (Q)
- `ObsIndex` is an increasing integer (0, 1, 2...) attached to the hourly data - this indicates a change in the observatory (e.g. of precise location) while the 3-letter IAGA code remained the same

Note that the **IAGA_code** variable is necessary in order to distinguish records from different observatories.

### Use `available_observatories` to find possible IAGA codes

We can get a dataframe containing the availability times of data from all the available observatories for a given collection (in this case hourly means):

In [ ]:
all_observatories = request.available_observatories("SW_OPER_AUX_OBSH2_", details=True)

Note the `Vires` returned a special object of type [pandas.DataFrame](https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html): 

In [ ]:
type(all_observatories)

This object is essentially a very advanced table with rich functional related to indexing, searching and extracting data. Think of it as an analogue of an Excel sheet, but in python. Let us just show part of it:

In [ ]:
all_observatories

Great, we have more than 200 ground observatories with their starting and end operation times.

We can also get a list of only the available observatories during a given time window:

In [ ]:
all_observatories = request.available_observatories("SW_OPER_AUX_OBSH2_", '2013-12-01', '2024-12-31', details=True)
all_observatories

### Plot observatories on a world map

In [ ]:
# pip install cartopy matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

proj = ccrs.Robinson()     # nice world projection; try PlateCarree() if you prefer
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=proj)
ax.set_global()

# base map
ax.add_feature(cfeature.LAND, facecolor="#f2f2f2")
ax.add_feature(cfeature.OCEAN, facecolor="#d8ecff")
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linewidth=0.3)
ax.gridlines(draw_labels=False, linewidth=0.3, alpha=0.5)

ax.scatter(all_observatories.Longitude, all_observatories.Latitude, marker='o', s=6, transform=ccrs.PlateCarree(), color="tab:red")

# Labels (nudge by a small lon/lat offset to reduce overlap)
label_dx = 2.0   # degrees longitude to nudge text
label_dy = 2.0   # degrees latitude to nudge text

for _, row in all_observatories.iterrows():
    label = str(row.get("site", ""))
    if label:
        ax.text(row["Longitude"] + label_dx,
                row["Latitude"] + label_dy,
                label,
                transform=ccrs.PlateCarree(),
                fontsize=8, weight="bold",
                zorder=4)

plt.tight_layout()
plt.show()

### European map

In [ ]:
# pip install cartopy matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

proj = ccrs.Robinson()     # nice world projection; try PlateCarree() if you prefer
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=proj)

ax.set_extent([-25, 45, 34, 72], crs=ccrs.PlateCarree())

# base map
ax.add_feature(cfeature.LAND, facecolor="#f2f2f2")
ax.add_feature(cfeature.OCEAN, facecolor="#d8ecff")
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linewidth=0.3)
ax.gridlines(draw_labels=False, linewidth=0.3, alpha=0.5)

ax.scatter(all_observatories.Longitude, all_observatories.Latitude, marker='o', s=6, transform=ccrs.PlateCarree(), color="tab:red")

# Labels (nudge by a small lon/lat offset to reduce overlap)
label_dx = 2.0   # degrees longitude to nudge text
label_dy = 2.0   # degrees latitude to nudge text

# labels
for _, row in all_observatories.iterrows():
    txt = ax.text(row["Longitude"], row["Latitude"], str(row["site"]),
                  transform=ccrs.PlateCarree(),
                  fontsize=8, weight="bold", zorder=4,
                  clip_on=True)               # <-- important
    # clip to the map’s outline (so nothing draws outside the projected boundary)
    try:
        txt.set_clip_path(ax.outline_patch)
    except Exception:
        pass  # fallback if Cartopy version lacks outline_patch

plt.tight_layout()
plt.show()

### Use `IAGA_code` to specify a particular observatory

Subset the collection with a special collection name like `"SW_OPER_AUX_OBSH2_:<IAGA_code>"` to get data from only that observatory:

In [ ]:
IAGA_code = "FUR"

request = SwarmRequest()
request.set_collection("SW_OPER_AUX_OBSH2_:" + IAGA_code)
request.set_products(measurements=["IAGA_code", "B_NEC"], 
                     auxiliaries=["Kp", "Dst"],
                     models=["MCO_SHA_2C", "MLI_SHA_2C"])
data = request.get_between("2014-01-01", "2023-12-31")

# Represent data as pandas.DataFrame, drop the empty column "Spacecraft" (obviously, the data is measured on the ground)
data = data.as_dataframe(expand=True)
# Rename B_NEC (North,East,Center) columns into X,Y,Z to match our convention from the lectures
data = data.rename(columns={f"B_NEC_{i}": j for i, j in zip("NEC", "XYZ")})
# Show the data
data

Next to the observatory data, we also fetched the corresponding core and lithospheric field values as predicted by the Comprehensive Inversion models. We will use these later.

### Plot time series

Let's plot the time series of all three components as a function of time using [matplotlib](https://matplotlib.org/stable/tutorials/pyplot.html) functionality.

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(15, 7), sharey=False, sharex=True)
fig.suptitle('Magnetic field at observatory ' + IAGA_code)
for i, component in enumerate("XYZ"):
    axes[i].plot(data.index, data[component],label=component)
    axes[i].set_ylabel(f"{component} (nT)")
    axes[i].grid()

Isn't it cool, few lines of python code and we have all geomagnetic data within the reach! Let's look into more...

## Removing the core and crustal field

The time series are dominated by the core field and also affected by the static crustal field. Therefore, before we can assume that the time-variations originate from the external source (e.g. magnetosphere) and corresponding induced components, we need to subtract the other sources, most importantly the core. 

To this end, we use modeled core field as given by the Comprehensive Inversion and subtract it from the data:

In [ ]:
B_obs = data[['X','Y','Z']].to_numpy()
B_core = data[['B_NEC_MCO_SHA_2C_N','B_NEC_MCO_SHA_2C_E','B_NEC_MCO_SHA_2C_C']].to_numpy()
B_crust = data[['B_NEC_MLI_SHA_2C_N','B_NEC_MLI_SHA_2C_E','B_NEC_MLI_SHA_2C_C']].to_numpy()

dB = B_obs - B_core - B_crust

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(15, 7), sharey=False, sharex=True)
fig.suptitle('Magnetic field variations at observatory ' + IAGA_code)
for i, component in enumerate("XYZ"):
    axes[i].plot(data.index, dB[:,i],label=component)
    axes[i].set_ylabel(f"{component} (nT)")
    axes[i].grid()

Some residual (non-modelled) constant offset is still visible, let's just detrend it:

In [ ]:
from scipy.signal import detrend

# Detrend along rows (each column separately)
dB = dB - np.nanmean(dB, axis=0)

In [ ]:
fig, axes = plt.subplots(nrows=3, figsize=(15, 7), sharey=False, sharex=True)
fig.suptitle('Magnetic field variations at observatory ' + IAGA_code)
for i, component in enumerate("XYZ"):
    axes[i].plot(data.index, dB[:,i],label=component)
    axes[i].set_ylabel(f"{component} (nT)")
    axes[i].grid()

# Coordinate transformation

Since the ring current is organized with respect to the geomagnetic dipole, we should transform the vector field and the coordinates of the observatory to geomagnetic coordinates. To this end, we will use the [dipole](https://github.com/klaundal/dipole/tree/main) package by Karl Laundal (just type `dipole.geo2mag?` to check the documentation of the function):

In [ ]:
import dipole

latitude = data['Latitude'].iloc[0] # Geographic latitude
longitude = data['Longitude'].iloc[0] # Geographic longitude

# Columns are North, East, Center
dB_mag = dB.copy()

dipole_2015 = dipole.Dipole(2015.) # IGRF epoch 2015

latitude_mag, longitude_mag, dB_mag[:,1], dB_mag[:,0] = dipole_2015.geo2mag(latitude, longitude, Ae = dB[:,1], An = dB[:,0])

In [ ]:
dipole_2015.geo2mag?

# Estimation of electromagnetic transfer function

Note that in the equation above, the magnetic field components $Z, H$ are expressed at an angular frequency $\omega$, but in reality we measure magnetic field in time. The relation between the vertical component time-series $z(t)$ its frequency domain is expressed by the Fourier integral (the expression for $H$ component is analogous):
$$
z(t)=\frac{1}{2\pi}\int\limits_{-\infty}^{\infty}Z(\omega)e^{\mathrm{i}\omega t}\mathrm{d}\omega,
$$

Next, we want to estimate a transfer function at a period of $T$ hours. To this end, we first split the hourly mean time-series into $L$ equal segments (also called realizations), each of length $T$ samples. For realization $l$, we perform the discrete Fourier transformation

$$
Z_l(\omega) = \sum_{k=1}^{T} \hat{Z}_{l,k} \exp(-\mathrm{i}\omega k),
$$

where $\hat{Z}_{l,k}$ is the $k$-th value in the $l$-th realization (segment), $\omega = 2\pi / T$. The same is done for the horizontal component $H$.

Then we use the least-squares spectral stacking method to estimate the $C$-response as

$$
C = - \frac{a}{2}\tan(\theta)\frac{\langle ZH^* \rangle }{\langle HH^* \rangle},
$$

where $H^*$ is the complex conjugate of $H$ and $\langle \cdots \rangle$ denotes the summation over $L$ realizations

$$
\langle ZH^* \rangle = \sum_{l=1}^L Z_l H_l^*
$$

As a quality measure of the estimated $C$-response, we use the [squared coherency](https://en.wikipedia.org/wiki/Coherence_(signal_processing)) defined as

$$
coh^2 = \frac{|\langle ZH^* \rangle|^2}{\langle ZZ^* \rangle\langle HH^* \rangle}
$$

Squared coherence is a number between 0 and 1. The high quality transfer function yields a value closer to a unity (provided all assumptions about the source geometry hold).

The uncertainty (error) of the least-squares estimate is given by

$$
\delta C^2 = |C|^2 (1 - coh^2) \frac{(1/\beta)^{1/L-1} - 1}{coh^2},
$$

where $1 - \beta$ is the probability that modulus $|C|$ lies within the error $|C| \pm \delta C$.

Below is the implementation of the method:

In [ ]:
def univariate_spectral_stacking(Z, H, Periods):
    """
    Univariate spectral stacking for Z/H sounding method
    Alexander Grayver, 2025
    
    Parameters
    ----------
    Z : 1D array-like (Nt,)
        Output-channel time series.
    H : 1D array-like (Nt,)
        Input-channel time series.
    Periods : 1D array-like (NPeriods,)
        Periods at which TFs are estimated.

    Returns
    -------
    TF : np.ndarray (NPeriods,)
        Estimated complex transfer function at each period.
    dTF : np.ndarray (NPeriods,)
        Standard error estimates.
    coh2 : np.ndarray (NPeriods,)
        Squared coherences.
    """
    
    Z = np.asarray(Z)
    H = np.asarray(H)
    Periods = np.asarray(Periods)
    
    TF_list = []
    dTF_list = []
    coh2_list = []

    for pidx, period in enumerate(Periods):
        
        length = int(np.round(period))

        H_f = []
        Z_f = []

        cfac = np.exp(-2j * np.pi * np.arange(length) / period)        
        for i in range(0, len(Z) - (length - 1), length):

            Z_seg = Z[i + np.arange(length)]
            H_seg = H[i + np.arange(length)]

            if not (np.isnan(Z_seg).any() or np.isnan(H_seg).any()):
                H_f.append(np.sum(H_seg * cfac))
                Z_f.append(np.sum(Z_seg * cfac))

        L = len(H_f)

        print(f"Period no. {pidx+1} of {len(Periods)}, {L} events")

        H_f = np.array(H_f, dtype=np.complex128)
        Z_f = np.array(Z_f, dtype=np.complex128)

        TF = np.vdot(H_f, Z_f) / np.vdot(H_f, H_f)
        TF_list.append(TF)

        # Squared coherence
        coh2 = np.abs(np.vdot(H_f, Z_f))**2 / (np.vdot(Z_f,Z_f)*np.vdot(H_f,H_f)).real
        coh2_list.append(coh2)

        # Uncertainty (Standard deviation) of the estimate
        beta = 0.95 # 1 - beta is the confidence interval, i.e. probability that the absolute unit value lies within error limits ±δTF.
        dTF = np.sqrt(((1. - coh2) * ((1/beta)**(1 / (L - 1)) - 1)) / coh2)
        dTF_list.append(dTF)

    TF = np.asarray(TF_list, dtype=np.complex128)
    dTF = np.asarray(dTF_list, dtype=float)
    coh2 = np.asarray(coh2_list, dtype=float)
    return TF, dTF, coh2

Specify inputs for data processng and call the spectral stacking function implemented above:

In [ ]:
T_start_h = 48;     # Shortest_period of the transfer function (>= Nyquist);
T_max_h   = 50*24;  # Longest of the desired transfer function;

Periods = np.logspace(np.log10(T_start_h),np.log10(T_max_h), 16);

# Note we used magnetic field in the transformed geomagnetic coordinates
[TF, dTF, coh2] = univariate_spectral_stacking(dB_mag[:,2], dB_mag[:,0], Periods);

Estimated transfer functions returned by the function **univariate_spectral_stacking** are just spectral ratios between vertical and horitonal components. We need to convert to the $C$-response following the equation above:

In [ ]:
a = 6371200 # Earth's mean radius
theta_mag = 90 - latitude_mag[0] # Geomagnetic co-latitude

C = - a / 2 * np.tan(np.deg2rad(theta_mag)) * TF
dC = np.abs(C) * dTF

Now the $C$-response and its uncertainty have units of **m**. For convenience, we plot them in **km**:

In [ ]:
fig, (ax_coh, ax_tf) = plt.subplots(2, 1, sharex=True, figsize=(8, 6),
                                    gridspec_kw={'height_ratios': [1, 2]})

# Coherence plot
ax_coh.plot(Periods/24, coh2, 'o-', color='tab:blue')
ax_coh.set_ylabel('Squared Coherence')
ax_coh.set_ylim(0, 1.05)
ax_coh.set_xscale('log')
ax_coh.grid(True)

# TF plot: real and imaginary parts
ax_tf.errorbar(Periods/24, C.real/1e3, yerr=dC/1e3, fmt='.', label='Real(C)', color='tab:orange')
ax_tf.errorbar(Periods/24, C.imag/1e3, yerr=dC/1e3, fmt='.', label='Imag(C)', color='tab:green')
ax_tf.set_xlabel('Period [day]')
ax_tf.set_ylabel('C [km]')
ax_tf.set_xscale('log')
ax_tf.grid(True)
ax_tf.legend()

fig.suptitle('C-response for observatory ' + IAGA_code)

plt.tight_layout()
plt.show()

# Estimation of the 1-D conductivity profile

We want to find a simple 1-D conductivity model that would explain the data. The 1-D model is given by layer thicknesses and conductivity values. In this exercise, we will seek only layer conductivities and and our model will consist of three layers. Since electrical conductities can vary over many orders of magnitude, we will invert for the log-conductivity. The model vector is then

$$
\mathbf{m} = [\log_{10}(m_1), \log_{10}(m_2), \log_{10}(m_3)]
$$

To find the conductivity values that minimize the misfit, we solve the following minimization problem:

$$
\min_{\mathbf{m}} \Phi_d(\mathbf{m}) + \alpha\Phi_m(\mathbf{m})
$$

Where $\Phi_d$ is the data misfit term given by

$$
\Phi_d(\mathbf{m}) = \left(C^{obs} - C^{mod}(\mathbf{m})\right)^H \Sigma_d^{-1} \left(C^{obs} - C^{mod}(\mathbf{m})\right),
$$

where $C^{obs}$ is the vector of estimated transfer functions, $C^{mod}(\mathbf{m})$ is the vector of predicted transfer functions. The subscript $H$ is complex-conjugate transpose. The data covariance matrix $\Sigma_d$ is a diagonal matrix with the estimated $C$-response uncertainties 

$$
\Sigma_d = \mathrm{diag}[\delta C_1, \dots, \delta C_{N_d}]
$$

The $\alpha$ is the regularization parameter and  $\Phi_m$ is the regularization term expressed as

$$
\Phi_m(\mathbf{m}) = \mathbf{m}^T\mathbf{m}
$$

Which minimizes the model norm, thereby preferring simpler models. 

Now we can implement the objective function:

In [ ]:
def phi(radii, sigma, periods, C_obs, dC_obs, regpar):
    C_mod, _, _, _ = c_utils.q_response_1D(periods, sigma, radii / 1e3, n = 1, kind = 'constant')
    C_mod *= 1e3 # km to m
    
    # Misfit
    residuals = (C_obs - C_mod) / dC_obs
    phi_d = np.vdot(residuals,residuals).real

    phi_m = np.linalg.norm(np.log10(sigma))**2
    
    return phi_d + regpar*phi_m

Next we define the three layer model with layer boundaries at the top and bottom of the Mantle Transition Zone (i.e. 410 and 660 km)

In [ ]:
dz = [410e3, 250e3, 2200e3] # thickness of the layers in m
radii = a - np.cumsum([0,] + dz)

Since we only have three parameters, we will seek the minimum by using the bruteforce (grid-search) method. That is, we will test log-spaced conductivity values within a range for each layer:

In [ ]:
sigma_values = np.logspace(-4, 1, 21)  # was 26
print(sigma_values)

To avoid bulky nested loops, we use the co-scalled defined using the product method. This method will create all combinations of values and we will be able to iterate over them within a single loop:

In [ ]:
from itertools import product
combinations = product(sigma_values, repeat=3)

Iterate over model combinations and compute the objective function, then store the result in an array:

In [ ]:
results = [(sigma, phi(radii, sigma, Periods*3600, C, dC, regpar=1000)) for sigma in combinations]

Once finished, we can see the model with the minimum misfit, corresponding to the model that best fits our observations:

In [ ]:
best_sigma, best_misfit = min(results, key=lambda t: t[1])

print("Lowest misfit model: ", best_sigma)
print("Lowest misfit value: ", best_misfit)

Now we can predict the responses for the best-fit model:

In [ ]:
C_best, _, _, _ = c_utils.q_response_1D(Periods*3600, best_sigma, radii / 1e3, n = 1, kind = 'constant')
C_best *= 1e3 # km to m

Finally, let's plot the observaed and modelled (best-fit model) responses together:

In [ ]:
fig, (ax_coh, ax_tf) = plt.subplots(2, 1, sharex=True, figsize=(8, 6),
                                    gridspec_kw={'height_ratios': [1, 2]})

# Coherence plot
ax_coh.plot(Periods/24, coh2, 'o-', color='tab:blue')
ax_coh.set_ylabel('Squared Coherence')
ax_coh.set_ylim(0, 1.05)
ax_coh.set_xscale('log')
ax_coh.grid(True)

# TF plot: real and imaginary parts
ax_tf.errorbar(Periods/24, C.real/1e3, yerr=dC/1e3, fmt='.', label='Real(C)', color='tab:orange')
ax_tf.errorbar(Periods/24, C.imag/1e3, yerr=dC/1e3, fmt='.', label='Imag(C)', color='tab:green')
ax_tf.plot(Periods/24, C_best.real/1e3, label='Best fit model', color='k')
ax_tf.plot(Periods/24, C_best.imag/1e3, label=None, color='k')
ax_tf.set_xlabel('Period [day]')
ax_tf.set_ylabel('C [km]')
ax_tf.set_xscale('log')
ax_tf.grid(True)
ax_tf.set_xlim(1, 60)
ax_tf.legend()

fig.suptitle('C-response for observatory ' + IAGA_code)

plt.tight_layout()
plt.show()

We can compare the models with the global electrical conductivity model from [Grayver et al. 2017](https://github.com/agrayver/ConductivityProfile):

In [ ]:
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

model_2017 = np.loadtxt('joint_model_Grayver2017.dat')

fig=plt.figure(figsize=(5, 6), facecolor='w', edgecolor='k')

plt.step(model_2017[:,1], model_2017[:,0]/1e3, where='pre', label = 'Global Model')

plt.step(np.r_[best_sigma, best_sigma[-1]], (a-radii)/1e3, where='pre', label = IAGA_code)

plt.plot([1e-4, 1e6], [410, 410], color='gray', linestyle='--', linewidth='1')
plt.plot([1e-4, 1e6], [660, 660], color='gray', linestyle='--', linewidth='1')

plt.xlim(1e-4, 1e2);
plt.ylim(0, 2300);
plt.xlabel(r'Conductivity $\sigma$ [S/m]', fontsize=14)
plt.ylabel(r'Depth [km]', fontsize=14)
plt.xscale('log')
plt.gca().invert_yaxis()
plt.legend()    
#plt.grid()

plt.tight_layout()
#plt.savefig('models.png', dpi=200)


## Exercise

- Vary the regularization parameter `regpar` and check how this changes the model and the data misfit. 
- Try estimating transfer function for longer and/or shorter periods. Which problems do you encounter (pay attention to the squared coherency and monotonic property of the $C$-response)?
- Try different mid-latitude observatories, for instance Boulder, US (BOU), or Alice Springs (ASP).
- Also try some high-latitude observatories: does the method still work? Are all underlying assumptions still hold? 

Note that the data processing methods used in research are a lot more advanced, and include robust statistics to mitigate outliers and non-gaussian noise, window tapering to lower the spectral leakage, and bias corrections. 